In [252]:
# !pip install -r requirements.txt
# !python -m spacy download en_core_web_sm

In [253]:
import pandas as pd 
import string
import re
import numpy as np
import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords 
from nltk.tokenize import TweetTokenizer
import emot
emot_obj = emot.emot()
import spacy

In [254]:
# !curl -L -o amazon.zip https://www.kaggle.com/api/v1/datasets/download/muhammadalaasar/amazon
# !unzip amazon.zip 

In [255]:
import pandas as pd
df = pd.read_csv('amazon.csv')


In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

- The dataset have 13 columns and 1465 observations. 

- actual_price , discounted_price , discounted_percentage , rating , rating_count should be numeric, continuos (float)

In [257]:
df.describe()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
count,1465,1465,1465,1465,1465,1465,1465,1463,1465,1465,1465,1465,1465,1465,1465,1465
unique,1351,1337,211,550,449,92,28,1143,1293,1194,1194,1194,1194,1212,1412,1465
top,B07JW9H4J1,"Fire-Boltt Ninja Call Pro Plus 1.83"" Smart Wat...",Computers&Accessories|Accessories&Peripherals|...,₹199,₹999,50%,4.1,"9,378",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AHIKJUDTVJ4T6DV6IUGFYZ5LXMPA,AE55KTFVNXYFD5FPY...","$@|\|TO$|-|,Sethu madhav,Akash Thakur,Burger P...","R3F4T5TRYPTMIG,R3DQIEC603E7AY,R1O4Z15FD40PV5,R...","Worked on iPhone 7 and didn’t work on XR,Good ...","I am not big on camera usage, personally. I wa...",https://m.media-amazon.com/images/I/413sCRKobN...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
freq,3,5,233,53,120,56,244,9,6,10,10,10,10,8,3,1


- describe does not work yet, since all are in object type

In [258]:
df.head(10)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
5,B08Y1TFSP6,pTron Solero TB301 3A Type-C Data and Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹149,"₹1,000",85%,3.9,"24,871",Fast Charging & Data Sync: Solero TB301 Type-C...,"AEQ2YMXSZWEOHK2EHTNLOS56YTZQ,AGRVINWECNY7323CW...","Jayesh,Rajesh k.,Soopy,amazon customer,Aman,Sh...","R7S8ANNSDPR40,R3CLZFLHVJU26P,RFF7U7MPQFUGR,R1M...","It's pretty good,Average quality,very good and...","It's a good product.,Like,Very good item stron...",https://m.media-amazon.com/images/I/31wOPjcSxl...,https://www.amazon.in/Solero-TB301-Charging-48...
6,B08WRWPM22,"boAt Micro USB 55 Tangle-free, Sturdy Micro US...",Computers&Accessories|Accessories&Peripherals|...,₹176.63,₹499,65%,4.1,"15,188",It Ensures High Speed Transmission And Chargin...,"AG7C6DAADCTRQJG2BRS3RIKDT52Q,AFU7BOMPVJ7Q3TTA4...","Vivek kumar,Amazon Customer,SARTHAK,Chiranjeev...","R8E73K2KWJRDS,RSD0JTIIWQQL8,R64CRSTE9SLW1,R2FR...","Long durable.,good,Does not charge Lenovo m8 t...",Build quality is good and it is comes with 2 y...,https://m.media-amazon.com/images/W/WEBP_4

In [259]:
df.duplicated().sum()

np.int64(0)

No dups?

In [260]:
df.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [261]:
df.drop(columns=['img_link','product_link', 'product_name','discount_percentage'], inplace=True)

- Image link and productlink ís not avaiavle any more, amazon have blocked them. So we decided to drop those col

- About product name, we can trace them back using product id or review id (key = review id)

- discount percentage could also be removed since it can be derived from discount price and actual price

In [262]:
df.head()

,product_id,category,discounted_price,actual_price,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content
0,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...
1,B098NS6PVG,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...
2,B096MSW6CT,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a..."
3,B08HDJ86NZ,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou..."
4,B08CF3B7N1,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th..."


In [263]:
unique_cate = df['category'].unique()
print(f"Number of unique categories: {len(unique_cate)}")

Number of unique categories: 211


### How to deal with category??

- also tokenize and keep unique

In [264]:
text = "Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables"

def tokenize_cate(cate):
    parts = re.split(r"[,&|\s]+", cate) # split comma, ampersand, or pipe or whitespace
    tokens = [p for p in parts if p]
    seen = set()
    unique_tokens = []
    for t in tokens:
        if t not in seen:
            seen.add(t)
            unique_tokens.append(t)
    return unique_tokens

print(tokenize_cate(text))


['Computers', 'Accessories', 'Peripherals', 'Cables', 'USBCables']


In [265]:

df['category'] = df["category"].apply(tokenize_cate)
df['category'].head()

0    [Computers, Accessories, Peripherals, Cables, ...
1    [Computers, Accessories, Peripherals, Cables, ...
2    [Computers, Accessories, Peripherals, Cables, ...
3    [Computers, Accessories, Peripherals, Cables, ...
4    [Computers, Accessories, Peripherals, Cables, ...
Name: category, dtype: object

## Deal with user name and user id

In [266]:
def remove_comma_and_tokenize(text):
    if isinstance(text, list):
        parts = " ".join(text)
    parts = re.split(",",text)
    return [p for p in parts if p]

user_sample = "Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jaspreet singh,Khaja moin,Anand,S.ARUMUGAM"
print(remove_comma_and_tokenize(user_sample))
    

['Manav', 'Adarsh gupta', 'Sundeep', 'S.Sayeed Ahmed', 'jaspreet singh', 'Khaja moin', 'Anand', 'S.ARUMUGAM']


In [267]:
# df['user_name'] = np.vectorize(remove_comma_and_tokenize)(df['user_name'])
df['user_name'] = df['user_name'].apply(remove_comma_and_tokenize)
df['user_name']

print(df['user_name'].dtype) 
print(df['user_name'].apply(type).value_counts())

object
user_name
<class 'list'>    1465
Name: count, dtype: int64


### Dealing with actual price and discount price

In [268]:
df['discounted_price'].unique()

array(['₹399', '₹199', '₹329', '₹154', '₹149', '₹176.63', '₹229', '₹499',
       '₹299', '₹219', '₹350', '₹159', '₹349', '₹13,999', '₹249',
       '₹13,490', '₹970', '₹279', '₹59', '₹11,499', '₹14,999', '₹179',
       '₹389', '₹599', '₹99', '₹899', '₹32,999', '₹209', '₹19,999',
       '₹999', '₹333', '₹507', '₹309', '₹1,199', '₹799', '₹6,999', '₹230',
       '₹649', '₹15,999', '₹348', '₹32,990', '₹139', '₹263', '₹7,999',
       '₹1,599', '₹26,999', '₹115', '₹10,901', '₹1,434', '₹7,299', '₹325',
       '₹29,999', '₹27,999', '₹30,990', '₹269', '₹24,999', '₹18,990',
       '₹290', '₹345', '₹1,099', '₹719', '₹849', '₹21,999', '₹449',
       '₹37,999', '₹7,390', '₹273.10', '₹15,990', '₹210', '₹1,299',
       '₹347', '₹228', '₹1,499', '₹8,499', '₹20,990', '₹9,999', '₹489',
       '₹23,999', '₹689', '₹1,699', '₹655', '₹749', '₹195', '₹416',
       '₹368', '₹29,990', '₹339', '₹15,490', '₹9,490', '₹637', '₹1,089',
       '₹12,499', '₹1,399', '₹88', '₹57.89', '₹205', '₹949', '₹379',
       '₹8,9

In [269]:

def handle_price(price):
    cleaned = re.sub(r'[₹,]', '', price) # r stand for regular expression
    try:
        return float(cleaned)
    except ValueError:
        return np.nan

df['discounted_price'] = df['discounted_price'].apply(lambda x: handle_price(x))
df['actual_price'] = df['actual_price'].apply(lambda x: handle_price(x))

In [270]:
df.isnull().count()

product_id          1465
category            1465
discounted_price    1465
actual_price        1465
rating              1465
rating_count        1465
about_product       1465
user_id             1465
user_name           1465
review_id           1465
review_title        1465
review_content      1465
dtype: int64

In [271]:


def text_normalize(text):
    #lowercasing
    text = text.lower()
    
    # hyperlinks removal
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    
    # remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Remove stopwords
    stopwords_txt = pd.read_csv('stop_words_english.csv', header=None)
    stop_words = stopwords_txt[0].tolist()
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)                                          # Space handling
    return text

# remove emoticons to using emot
def handle_emoticons(text, remove_emoticon=True):
    dict_emoticons = dict(zip(emot_obj.emoticons(text)['value'], emot_obj.emoticons(text)['mean']))
    res_emoticons =  dict(sorted(dict_emoticons.items(), key = lambda kv:len(kv[1]), reverse=True))
    for emoticon, mean in res_emoticons.items():
        if remove_emoticon:
            text = text.replace(emoticon, "")
        else:
            text = text.replace(emoticon, mean)
    return text

def handle_emojis(text, remove_emoji=True):
    for emoji, mean in zip(emot_obj.emoji(text)['value'], emot_obj.emoji(text)['mean']):
        if remove_emoji:
            text = text.replace(emoji, "")
        else:
            text = text.replace(emoji, mean)
    return text

def keep_ascii_alnum(text):
    # 1) Strip out any char that's not a letter, digit, or whitespace
    cleaned = re.sub(r'[^A-Za-z0-9\s]+', '', text)
    
    # 2) Collapse runs of whitespace into a single space, and trim ends
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    return str(cleaned)

In [272]:
df['title and review'] = df['review_title'] + ' ' + df['review_content']

# Combine reivew_title and review_content into 1 col
# df.drop(columns=['review_title', 'review_content'], inplace=True)
df['title and review'] = df['title and review'].astype(str)

In [273]:
# apply functions
# df['title and review'] = df['title and review'].apply(lambda x: text_normalize(x))
# df['title and review'] = df['title and review'].apply(lambda x: handle_emoticons(x))
# df['title and review'] = df['title and review'].apply(lambda x: handle_emojis(x))
# df['title and review'] = df['title and review'].apply(lambda x: keep_ascii_alnum(x))
# too slow

df['title and review'] = np.vectorize(text_normalize)(df['title and review'])
df['title and review'] = np.vectorize(handle_emoticons)(df['title and review'])
df['title and review'] = np.vectorize(handle_emojis)(df['title and review'])
df['title and review'] = np.vectorize(keep_ascii_alnum)(df['title and review'])

In [274]:
print(df['review_title'][6] + ' ' + df['review_content'][6])

Long durable.,good,Does not charge Lenovo m8 tab,Best charging cable,good,Boat,Product was good,1.5 m का केबल मेरे लिए बहुत ही लाभदायक है । Build quality is good and it is comes with 2 year warranty.,Good Product,Bought it for charging my mobile & tab but it doesn't work for Lenovo m8 tab,Guys this cable is is best compare of everyone over heat protection, quickly charging, but chance to shock circuit,good,Nice,Good quality,1.5एम का डाटा केबल मेरे लिए बहुत ही लाभदायक है ।  मैं इस केबल को लाइन में चार्ज करते समय फोन बहुत आराम से उपयोग  कर पा रहा हु । आप इस केबल से 15watt का  चार्जर उसे कर सकते है (इससे जड़ा नही) । लोकल बाजार में इसका दाम 150 है ,  अमेजन पे ये केबल मुझे 67 में मिला । गर्व से कहो हम हिंदू है , जय हिंद जय भारत ,


In [275]:
print(df['title and review'][6])

long durable good charge lenovo m8 tab charging cable good boat product good 1 5 build quality good 2 year warranty good product bought charging mobile tab doesn work lenovo m8 tab guys cable compare heat protection charging chance shock circuit good nice good quality 1 5 15watt 150 67


In [276]:
df.drop(columns=['review_title','review_content'])

,product_id,category,discounted_price,actual_price,rating,rating_count,about_product,user_id,user_name,review_id,title and review
0,B07JW9H4J1,"[Computers, Accessories, Peripherals, Cables, ...",399.0,1099.0,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","[Manav, Adarsh gupta, Sundeep, S.Sayeed Ahmed,...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...",satisfied charging fast money product review g...
1,B098NS6PVG,"[Computers, Accessories, Peripherals, Cables, ...",199.0,349.0,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","[ArdKn, Nirbhay kumar, Sagar Viswanathan, Asp,...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...",good braided cable type device good quality pr...
2,B096MSW6CT,"[Computers, Accessories, Peripherals, Cables, ...",199.0,1899.0,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","[Kunal, Himanshu, viswanath, sai niharka, saqi...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...",good speed earlier versions good product worki...
3,B08HDJ86NZ,"[Computers, Accessories, Peripherals, Cables, ...",329.0,699.0,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","[Omkar dhale, JD, HEMALATHA, Ajwadh a., amar s...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...",good product good nice nice product time chang...
4,B08CF3B7N1,"[Computers, Accessories, Peripherals, Cables, ...",154.0,399.0,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","[rahuls6099, Swasat Borah, Ajay Wadke, Pranali...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...",good original decent good secondary quality go...
...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,"[Home, Kitchen, HomeAppliances, WaterPurifiers...",379.0,919.0,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","[Prabha ds, Raghuram bk, Real Deal, Amazon Cus...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...",received product spanner excellent product sat...
1461,B01M6453MB,"[Home, Kitchen, HomeAppliances, SmallKitchenAp...",2280.0,3045.0,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","[Manu Bhai, Naveenpittu, Evatira Sangma, JAGAN...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...",good couldn return bcoz needed emergency don b...
1462,B009P2LIL4,"[Home, Kitchen, Heating, Cooling, AirQuality, ...",2219.0,3080.0,3.6,468,International design and styling|Two heat sett...,"AGVPWCMAHYQWJOQKMUJN4DW3KM5Q,AF4Q3E66MY4SR7YQZ...","[Nehal Desai, Danish Parwez, Amazon Customer, ...","R1TLRJVW4STY5I,R2O455KRN493R1,R3Q5MVGBRIAS2G,R...",good work melt 2 month good durable review def...
1463,B00J5DYCCA,"[Home, Kitchen, Heating, Cooling, AirQuality, ...",1399.0,1890.0,4,"8,031",Fan sweep area: 230 MM ; Noise level: (40 - 45...,"AF2JQCLSCY3QJATWUNNHUSVUPNQQ,AFDMLUXC5LS5RXDJS...","[Shubham Dubey, E.GURUBARAN, Mayank S., eusuf ...","R39Q2Y79MM9SWK,R3079BG1NIH6MB,R29A31ZELTZNJM,R...",fan speed slow good quality good product good ...


## Lemmatization

In [277]:
# source: https://www.geeksforgeeks.org/python-lemmatization-with-nltk/

# Lemmatization
nlp = spacy.load("en_core_web_sm")  

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

sample_text = "The quick brown foxes are jumping over the lazy dogs."
print("lemmatized text:", lemmatize_text(sample_text))



lemmatized text: the quick brown fox be jump over the lazy dog .


In [278]:
df['title and review'] = np.vectorize(lemmatize_text)(df['title and review'])
df['title and review'][6]

'long durable good charge lenovo m8 tab charge cable good boat product good 1 5 build quality good 2 year warranty good product buy charge mobile tab doesn work lenovo m8 tab guy cable compare heat protection charge chance shock circuit good nice good quality 1 5 15watt 150 67'

In [279]:
df['title and review'].head(10)

0    satisfied charge fast money product review goo...
1    good braid cable type device good quality prod...
2    good speed early version good product work goo...
3    good product good nice nice product time chang...
4    good original decent good secondary quality go...
5    pretty good average quality good usb cable goo...
6    long durable good charge lenovo m8 tab charge ...
7    worth money suitable android auto good product...
8    work linux model antenna brand less price easy...
9    good braid cable type device good quality prod...
Name: title and review, dtype: object

## Tokenize the col 'title and review' (omit for now)

In [280]:
# tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
#                                reduce_len=True)

# # df['title and review'] = np.vectorize(tokenizer.tokenize)(df['title and review'])
# df['title and review'] = df['title and review'].apply(tokenizer.tokenize)

 - preserve case = false -> already did
 - strip handle = tru: @mention -> mention
 - reduce len: haaaa -> ha

In [281]:
# df['title and review'][15]

In [282]:
# df['title and review'].head(10)

## Final remark
- output: amazon-clean.csv
- output: dic-userid-username.json ???
- output: dic-reviewid-userid (primary key will be review_id)

In [283]:
df.to_csv("amazon-clean.csv", index = False)